In [1]:
import pandas as pd
import solution_1 as solution
from lightgbm import LGBMRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
train = pd.read_csv('datasets/Train.csv')
test = pd.read_csv('datasets/Test.csv')
ss = pd.read_csv('datasets/SampleSubmission.csv')

In [3]:
train.head()

,ward,total_households,total_individuals,target,dw_00,dw_01,dw_02,dw_03,dw_04,dw_05,...,pw_03,pw_04,pw_05,pw_06,pw_07,pw_08,ADM4_PCODE,lat,lon,NL
0,41601001: Ward 1,1674.45058,5888.20750,16.773757,0.933841,0.000846,0.005490,0.000676,0.000000,0.001372,...,0.002848,0.007537,0.000000,0.012928,0,0,ZA4161001,-29.682270,24.734743,0.292039
1,41601002: Ward 2,1736.99230,6735.33812,21.496661,0.696940,0.001253,0.004402,0.000000,0.002301,0.001323,...,0.014566,0.057127,0.019092,0.004131,0,0,ZA4161002,-29.119311,24.757737,3.207775
2,41601003: Ward 3,2403.57591,7273.04995,10.931425,0.810545,0.004517,0.008891,0.003986,0.007735,0.000956,...,0.057560,0.010358,0.001421,0.040881,0,0,ZA4161003,-29.142276,25.094093,0.000000
3,41601004: Ward 4,1740.78737,5734.49046,23.119257,0.659914,0.000000,0.006129,0.000000,0.000813,0.037245,...,0.000000,0.000669,0.000000,0.005011,0,0,ZA4161004,-29.372052,24.942867,2.038778
4,41601005: Ward 5,1730.51451,6657.23835,13.652252,0.950575,0.000655,0.001473,0.000598,0.006999,0.000818,...,0.004859,0.001290,0.000673,0.017629,0,0,ZA4161005,-29.409381,25.290165,0.000000


In [4]:
train.columns.values

array(['ward', 'total_households', 'total_individuals', 'target', 'dw_00',
       'dw_01', 'dw_02', 'dw_03', 'dw_04', 'dw_05', 'dw_06', 'dw_07',
       'dw_08', 'dw_09', 'dw_10', 'dw_11', 'dw_12', 'dw_13', 'psa_00',
       'psa_01', 'psa_02', 'psa_03', 'psa_04', 'stv_00', 'stv_01',
       'car_00', 'car_01', 'lln_00', 'lln_01', 'lan_00', 'lan_01',
       'lan_02', 'lan_03', 'lan_04', 'lan_05', 'lan_06', 'lan_07',
       'lan_08', 'lan_09', 'lan_10', 'lan_11', 'lan_12', 'lan_13',
       'lan_14', 'pg_00', 'pg_01', 'pg_02', 'pg_03', 'pg_04', 'lgt_00',
       'pw_00', 'pw_01', 'pw_02', 'pw_03', 'pw_04', 'pw_05', 'pw_06',
       'pw_07', 'pw_08', 'ADM4_PCODE', 'lat', 'lon', 'NL'], dtype=object)

In [5]:
train.dtypes

ward                  object
total_households     float64
total_individuals    float64
target               float64
dw_00                float64
                      ...   
pw_08                  int64
ADM4_PCODE            object
lat                  float64
lon                  float64
NL                   float64
Length: 63, dtype: object

In [6]:
cat_cols = [col for col in train.columns if train[col].dtype == 'object']
cat_cols

['ward', 'ADM4_PCODE']

In [7]:
train[cat_cols].describe()

,ward,ADM4_PCODE
count,2822,2822
unique,2822,2822
top,41601001: Ward 1,ZA4161001
freq,1,1


In [8]:
label_train = train.drop('ADM4_PCODE', axis = 1)
label_train = train.dropna(axis = 1)
label_train 

,ward,total_households,total_individuals,target,dw_00,dw_01,dw_02,dw_03,dw_04,dw_05,...,pw_03,pw_04,pw_05,pw_06,pw_07,pw_08,ADM4_PCODE,lat,lon,NL
0,41601001: Ward 1,1674.45058,5888.20750,16.773757,0.933841,0.000846,0.005490,0.000676,0.000000,0.001372,...,0.002848,0.007537,0.000000,0.012928,0,0,ZA4161001,-29.682270,24.734743,0.292039
1,41601002: Ward 2,1736.99230,6735.33812,21.496661,0.696940,0.001253,0.004402,0.000000,0.002301,0.001323,...,0.014566,0.057127,0.019092,0.004131,0,0,ZA4161002,-29.119311,24.757737,3.207775
2,41601003: Ward 3,2403.57591,7273.04995,10.931425,0.810545,0.004517,0.008891,0.003986,0.007735,0.000956,...,0.057560,0.010358,0.001421,0.040881,0,0,ZA4161003,-29.142276,25.094093,0.000000
3,41601004: Ward 4,1740.78737,5734.49046,23.119257,0.659914,0.000000,0.006129,0.000000,0.000813,0.037245,...,0.000000,0.000669,0.000000,0.005011,0,0,ZA4161004,-29.372052,24.942867,2.038778
4,41601005: Ward 5,1730.51451,6657.23835,13.652252,0.950575,0.000655,0.001473,0.000598,0.006999,0.000818,...,0.004859,0.001290,0.000673,0.017629,0,0,ZA4161005,-29.409381,25.290165,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2817,34503001: Ward 1,3261.68067,11551.56486,6.393480,0.874972,0.003146,0.017230,0.004735,0.048888,0.000654,...,0.000472,0.000472,0.000000,0.006203,0,0,ZA3453001,-27.675590,23.057920,9.118927
2818,34503002: Ward 2,1463.06432,7908.90411,15.668000,0.636235,0.002262,0.010433,0.000000,0.003030,0.000000,...,0.003030,0.000713,0.000000,0.008414,0,0,ZA3453002,-27.591408,22.907161,1.697395
2819,34503003: Ward 3,1395.18881,6242.37527,13.238795,0.855975,0.006601,0.061118,0.000812,0.003912,0.000812,...,0.002150,0.001458,0.000000,0.003670,0,0,ZA3453003,-27.952550,22.749698,2.792501
2820,34503004: Ward 4,1679.66964,8014.73089,17.639523,0.792552,0.004423,0.016099,0.000000,0.000674,0.000000,...,0.009235,0.007870,0.000000,0.004803,0,0,ZA3453004,-27.913875,22.755314,2.793699


In [9]:
X = label_train.drop(['target'], axis = 1)
y = label_train.target

X = solution.prepare_data(X)

X.shape, y.shape

((2822, 62), (2822,))

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train

,ward,total_households,total_individuals,dw_00,dw_01,dw_02,dw_03,dw_04,dw_05,dw_06,...,pw_03,pw_04,pw_05,pw_06,pw_07,pw_08,ADM4_PCODE,lat,lon,NL
4,1730.51451,6657.23835,0.950575,0.000655,0.001473,0.000598,0.006999,0.000818,0.004985,0.009150,...,0.001290,0.000673,0.017629,0.0,0.0,-29.409381,25.290165,0.000000,180.0,180.0
989,1566.75097,9579.48818,0.818528,0.144007,0.003979,0.000653,0.000000,0.001377,0.018032,0.002282,...,0.001149,0.001343,0.127946,0.0,0.0,-28.191182,32.282169,4.212427,883.0,883.0
636,22986.82397,46238.77145,0.088837,0.001869,0.009969,0.001874,0.002164,0.003917,0.022038,0.331408,...,0.019220,0.013063,0.002908,0.0,0.0,-25.924032,28.017629,39.056824,1874.0,1874.0
1484,6355.13719,29969.11015,0.727781,0.182893,0.003878,0.000970,0.018168,0.000342,0.004435,0.023577,...,0.001276,0.024590,0.048725,0.0,0.0,-30.013665,30.864133,40.579246,1191.0,1191.0
145,2641.99174,8795.84951,0.798339,0.136885,0.003319,0.000000,0.000000,0.001351,0.000943,0.013488,...,0.011775,0.007435,0.039754,0.0,0.0,-28.581655,28.771377,16.897821,321.0,321.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1079,1180.04517,7150.24808,0.494800,0.471627,0.007606,0.001714,0.001525,0.000000,0.002646,0.003659,...,0.010370,0.000000,0.746162,0.0,0.0,-28.845459,31.712232,4.908893,959.0,959.0
2664,1482.60203,5624.03726,0.903038,0.001037,0.026814,0.005368,0.002075,0.001468,0.021972,0.013154,...,0.000764,0.000000,0.000000,0.0,0.0,-31.469394,19.764759,4.560616,18.0,18.0
1599,2162.61387,8768.99223,0.957339,0.033177,0.002298,0.000498,0.000000,0.000539,0.000000,0.002254,...,0.001627,0.000473,0.004666,0.0,0.0,-23.877147,31.081128,13.300234,2474.0,2474.0
516,8796.82506,28630.59995,0.740526,0.005934,0.046746,0.010310,0.011097,0.003603,0.020424,0.044000,...,0.025029,0.000465,0.002667,0.0,0.0,-26.298141,28.470749,40.612122,1754.0,1754.0


In [11]:
y_train

4       13.652252
989     33.173717
636     17.481900
1484    22.619954
145     33.255493
          ...    
1079    37.181816
2664    13.398832
1599    28.903997
516      8.554258
357     10.415360
Name: target, Length: 2116, dtype: float64

In [12]:
lgbm = LGBMRegressor()
bagging_regressor = BaggingRegressor(lgbm)
model = RandomForestRegressor()

In [13]:
model.fit(X_train, y_train)

RandomForestRegressor()

In [14]:
bagging_regressor.fit(X_train, y_train)

BaggingRegressor(base_estimator=LGBMRegressor())

In [15]:
from sklearn.metrics import mean_squared_error

In [16]:
preds = model.predict(X_test)
bag_preds = bagging_regressor.predict(X_test)

In [43]:
print(mean_squared_error(preds, y_test))
print(mean_squared_error(bag_preds, y_test))

13.280139112878244
12.025181073937164


In [29]:
import math
print(math.sqrt(mean_squared_error(preds, y_test)))
print(math.sqrt(mean_squared_error(bag_preds, y_test)))

3.3802381343995944
3.23938723027138


In [30]:
#test = test.drop(cat_cols, axis = 1)
test = solution.prepare_data(test)
test

,ward,total_households,total_individuals,dw_00,dw_01,dw_02,dw_03,dw_04,dw_05,dw_06,...,pw_03,pw_04,pw_05,pw_06,pw_07,pw_08,ADM4_PCODE,lat,lon,NL
0,2504.95194,8745.15151,0.947257,0.000873,0.002021,0.000000,0.000000,0.030116,0.000452,0.013018,...,0.000000,0.000000,0.006649,0.0,0.0,-32.637758,23.848688,0.000000,0.382411,0.382411
1,2080.27718,7258.11764,0.844993,0.000481,0.043629,0.004714,0.012323,0.012300,0.022132,0.022412,...,0.000000,0.000000,0.002916,0.0,0.0,-31.990536,24.555818,0.000000,0.383399,0.383399
2,1106.62639,5919.13170,0.651380,0.007937,0.007113,0.000000,0.001977,0.259711,0.006505,0.044153,...,0.000000,0.000000,0.000000,0.0,0.0,-32.283595,24.563940,8.269556,0.384387,0.384387
3,2175.56096,10280.57452,0.410837,0.002468,0.011511,0.000485,0.000000,0.449604,0.009256,0.101963,...,0.000661,0.000000,0.001379,0.0,0.0,-32.261612,24.542202,8.626625,0.385375,0.385375
4,1270.83883,6018.34202,0.942851,0.002638,0.000821,0.000000,0.000891,0.000787,0.000830,0.027930,...,0.000000,0.000000,0.001660,0.0,0.0,-32.251571,24.558537,8.601754,0.386364,0.386364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008,16076.89903,43296.51372,0.668295,0.002827,0.207749,0.028813,0.069741,0.010701,0.003941,0.000804,...,0.000291,0.000148,0.002277,0.0,0.0,-33.806524,18.496094,33.913055,0.377470,0.377470
1009,17470.24004,53538.12816,0.483111,0.000824,0.001189,0.000870,0.000554,0.000370,0.001954,0.088044,...,0.008313,0.000348,0.022417,0.0,0.0,-33.982120,18.673308,60.009486,0.378458,0.378458
1010,9467.25336,40332.11708,0.579036,0.007184,0.066458,0.001003,0.000371,0.150977,0.007699,0.099672,...,0.000990,0.000352,0.010670,0.0,0.0,-34.062704,18.767457,35.834860,0.379447,0.379447
1011,6111.31195,27793.92916,0.623537,0.002216,0.101188,0.001517,0.000939,0.158979,0.013516,0.076050,...,0.000000,0.000204,0.026741,0.0,0.0,-34.057772,18.487893,50.153301,0.380435,0.380435


In [31]:
predictions = bagging_regressor.predict(test)

In [32]:
sub_file = ss.copy()
sub_file['target'] = predictions

In [33]:
sub_file.to_csv('6th_submission.csv', index = False)

In [23]:
sub_file

,ward,target
0,21001001: Ward 1,19.096535
1,21001002: Ward 2,14.151547
2,21001003: Ward 3,21.620903
3,21001004: Ward 4,17.689307
4,21001005: Ward 5,21.852446
...,...,...
1008,19100107: Ward 107,7.332830
1009,19100108: Ward 108,22.371736
1010,19100109: Ward 109,17.823892
1011,19100110: Ward 110,13.915760
